In [1]:
import pandas as pd
import numpy as np

In [2]:
df_exp = pd.read_csv("experimental_microstates.csv")
df_exp


,Microstate ID of A,Microstate ID of HA,Molecule ID,pKa (exp),pKa SEM (exp),pKa ID,Microstate identification source
0,SM07_micro004,SM07_micro006,SM07,6.08,0.01,SM07_pKa1,NMR measurement
1,SM14_micro001,SM14_micro002,SM14,5.30,0.01,SM14_pKa2,NMR measurement
2,SM14_micro002,SM14_micro003,SM14,2.58,0.01,SM14_pKa1,NMR measurement
3,SM02_micro002,SM02_micro004,SM02,5.03,0.01,SM02_pKa1,Estimated based on SM07 NMR measurement
4,SM04_micro003,SM04_micro002,SM04,6.02,0.01,SM04_pKa1,Estimated based on SM07 NMR measurement
5,SM09_micro003,SM09_micro001,SM09,5.37,0.01,SM09_pKa1,Estimated based on SM07 NMR measurement
6,SM12_micro012,SM12_micro006,SM12,5.28,0.01,SM12_pKa1,Estimated based on SM07 NMR measurement
7,SM13_micro005,SM13_micro001,SM13,5.77,0.01,SM13_pKa1,Estimated based on SM07 NMR measurement
8,SM15_micro004,SM15_micro002,SM15,8.94,0.01,SM15_pKa2,Estimated based on SM14 NMR measurement
9,SM15_micro002,SM15_micro003,SM15,4.70,0.01,SM15_pKa1,Estimated based on SM14 NMR measurement


In [3]:
df_col = pd.read_csv("typeI_submission_collection.csv")
df_col.head()

FileNotFoundError: File b'typeI_submission_collection.csv' does not exist

In [ ]:
df_col[df_col["Molecule ID"] == "SM14"]

In [ ]:
df_exp_microstates = df_exp.rename(columns={'Microstate ID of A': 'Exp. Microstate ID of A', 'Microstate ID of HA': 'Exp. Microstate ID of HA'})
df_exp_microstates = df_exp_microstates.drop(columns=['Molecule ID', "pKa (exp)", 'pKa SEM (exp)'])
df_exp_microstates

1. Add experimental microstates to collection
2. Drop entries of molecules with unknown experimental microstate
3. check for matches and give a score

In [ ]:
df_col_ms = pd.merge(df_col, df_exp_microstates, on="pKa ID", how="outer" )
# Drop rows whose "Exp. Microstate ID of A" column has NaN value
df_col_ms = df_col_ms.dropna(subset=["Exp. Microstate ID of A"])
# Drop rows whose "receipt_id" column has NaN value
#df_col_ms = df_col_ms.dropna(subset=["receipt_id"])
df_col_ms = df_col_ms.reset_index()
df_col_ms = df_col_ms.drop(columns=['index'])
df_col_ms.tail()

In [ ]:
df_col_ms[df_col_ms["Molecule ID"] == "SM14"]

In [ ]:
# Add a score column based on microstate ID match

df_col_ms["Microstate pair match"] = None

for i, row in enumerate(df_col_ms.iterrows()):
    pred_ms_A = row[1]["Microstate ID of A"]
    pred_ms_HA = row[1]["Microstate ID of HA"]
    print("{} and {}".format(exp_ms_A, exp_ms_HA))
    pred_ms_pair = str(pred_ms_A) + " " + str(pred_ms_HA)
    
    exp_ms_A = row[1]["Exp. Microstate ID of A"]
    exp_ms_HA = row[1]["Exp. Microstate ID of HA"]
    exp_ms_pair = str(exp_ms_A) + " " + str(exp_ms_HA)
    
    if pred_ms_pair == exp_ms_pair:
        df_col_ms.loc[i,"Microstate pair match"] = 1
        print(pred_ms_pair )
        print(exp_ms_pair)
        print("1")
    else:
        df_col_ms.loc[i,"Microstate pair match"] = 0    
        print(pred_ms_pair )
        print(exp_ms_pair)
        print("0")
               

In [ ]:
df_col_ms.head()

In [ ]:
submission_IDs = list(set(df_col_ms["receipt_id"].values))
#submission_IDs = list(set(df_col_ms["receipt_id"].to_list()))

submission_IDs 

In [ ]:
microstate_pair_relative_accuracy = []
number_of_known_microstate_pairs = 10

for submission_ID in submission_IDs:
    # Take subset of df_col_ms based on submission ID
    df_submission = df_col_ms[df_col_ms["receipt_id"] == submission_ID]
    # Count how many times "Microstate pair match" column has value 1
    pair_match = df_submission["Microstate pair match"].values
    relative_accuracy = (np.sum(pair_match)/number_of_known_microstate_pairs)*100
    print("Relative accuracy of {}:".format(submission_ID), relative_accuracy)
    microstate_pair_relative_accuracy.append(relative_accuracy)
   
microstate_pair_relative_accuracy

In [ ]:
data = {'Submission ID': submission_IDs, 'Relative accuracy of microstate pairs(%)': microstate_pair_relative_accuracy}
df_rel_acc = pd.DataFrame.from_dict(data)

df_rel_acc = df_rel_acc.sort_values(by="Relative accuracy of microstate pairs(%)", ascending=False) #.reset_index(inplace=True)
df_rel_acc

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams["figure.figsize"] = [4,4]

In [ ]:
def barplot(df, x_label, y_label, title):
    """Creates bar plot of a given dataframe.

    Args:
        df: Pandas Dataframe that should have columns with columnnames specified in other arguments.
        x_label: str, column name of x axis categories
        y_label: str, column name of y axis values
        title: str, the title of the plot

    """
    # Plot style
    plt.close()
    plt.style.use(["seaborn-talk", "seaborn-whitegrid"])
    plt.rcParams['axes.labelsize'] = 18
    plt.rcParams['xtick.labelsize'] = 14
    plt.rcParams['ytick.labelsize'] = 16
    plt.ylim(0,100)
    #plt.tight_layout()

    # Plot
    data = df
    x = range(len(data[y_label]))
    y = data[y_label]
    plt.bar(x, y)
    plt.xticks(x, data[x_label], rotation=90)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    if len(title) > 70:
        plt.title(title[:70]+"...")
    else:
        plt.title(title)
    plt.tight_layout()

In [ ]:
barplot(df = df_rel_acc, 
        x_label = "Submission ID", 
        y_label = "Relative accuracy of microstate pairs(%)", 
        title = "Relative accuracy of microstate pairs in Type I submissions" )
plt.savefig("TypeI_microstate_pair_relative_accuracy.pdf")

## Analyze relative microstate pair accuracy for each molecule

In [ ]:
df_col_ms.head()

In [ ]:
mol_IDs = list(set(df_col_ms["Molecule ID"].values))
mol_IDs